In [135]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [136]:
import sys
sys.path.append('/ccs/home/pstjohn/uniparc_modeling')

from bert.go import Ontology
ont = Ontology()

In [159]:
ont.total_nodes

32012

In [137]:
import os
dg_dir = '/ccs/home/pstjohn/project_work/deepgreen'
seq_df = pd.read_parquet(os.path.join(dg_dir, 'setaria.parquet'))
go_results = np.load('/ccs/home/pstjohn/member_work/setaria_go.npz')
dg_accessions = np.load('deepgreen_accessions.npz')['dg_list']

In [138]:
go_df = pd.DataFrame(np.vstack([go_results['protein'], go_results['go']]).T, columns=['protein_idx', 'go_idx'])

go_df = go_df.merge(seq_df['accession'], left_on='protein_idx', right_index=True)\
    .merge(pd.Series(ont.term_index, name='GO'), left_on='go_idx', right_index=True).sort_index()

In [139]:
go_df.head()

,protein_idx,go_idx,accession,GO
0,0,6,A0A0U2HKN4,GO:0008150
1,0,53,A0A0U2HKN4,GO:0005488
2,0,67,A0A0U2HKN4,GO:0003723
3,0,70,A0A0U2HKN4,GO:0043604
4,0,71,A0A0U2HKN4,GO:1901566


In [140]:
from scipy.stats import hypergeom

In [141]:
go_df.accession.isin(dg_accessions).sum()

20581

In [160]:
go_df.head()

,protein_idx,go_idx,accession,GO
0,0,6,A0A0U2HKN4,GO:0008150
1,0,53,A0A0U2HKN4,GO:0005488
2,0,67,A0A0U2HKN4,GO:0003723
3,0,70,A0A0U2HKN4,GO:0043604
4,0,71,A0A0U2HKN4,GO:1901566


In [142]:
len(go_counts_background)

3801

In [143]:
go_counts_background = go_df.GO.value_counts()
go_counts_deepgreen = go_df[go_df.accession.isin(dg_accessions)].GO.value_counts()

* M: total number of background genes
* n: number of background genes with given GO annotation
* N: total number of selected genes
* x: number of selected genes with GO annotation

In [144]:
merged_go_counts = pd.DataFrame(go_counts_deepgreen).merge(
    go_counts_background, left_index=True, right_index=True,
    how='left', suffixes=['_dg', '_bg'])

M = len(go_df.accession.unique())  # Total number of background genes
N = len(dg_accessions)

merged_go_counts['p_val'] = merged_go_counts.apply(
    lambda row: hypergeom.sf(row.GO_dg - 1, M, row.GO_bg, N, loc=0), 1)

merged_go_counts['fold_enrichment'] = merged_go_counts.apply(
    lambda row: (row.GO_dg / N) / (row.GO_bg / M), 1)

merged_go_counts['significant'] = merged_go_counts.p_val < (0.05 / len(merged_go_counts))
merged_go_counts.sort_values('p_val').head()

,GO_dg,GO_bg,p_val,fold_enrichment,significant
GO:0031224,379,8302,1.695433e-47,1.994694,True
GO:0016021,379,8302,1.695433e-47,1.994694,True
GO:0016020,399,9415,1.493978e-42,1.851708,True
GO:0005886,146,2284,9.164595e-30,2.793037,True
GO:0031090,185,3488,8.154656e-28,2.317475,True


In [149]:
go_descriptions = pd.DataFrame.from_dict(dict(ont.G.nodes(data=True)), orient='index').drop('index', 1)

In [151]:
merged_go_counts_with_descriptions = merged_go_counts.merge(go_descriptions, left_index=True, right_index=True, how='left')

In [158]:
merged_go_counts_with_descriptions.sort_values('p_val').head(30)

,GO_dg,GO_bg,p_val,fold_enrichment,significant,name,namespace
GO:0031224,379,8302,1.695433e-47,1.994694,True,intrinsic component of membrane,cellular_component
GO:0016021,379,8302,1.695433e-47,1.994694,True,integral component of membrane,cellular_component
GO:0016020,399,9415,1.493978e-42,1.851708,True,membrane,cellular_component
GO:0005886,146,2284,9.164595e-30,2.793037,True,plasma membrane,cellular_component
GO:0031090,185,3488,8.154656e-28,2.317475,True,organelle membrane,cellular_component
GO:0046873,43,216,1.118942e-27,8.698304,True,metal ion transmembrane transporter activity,molecular_function
GO:0071805,25,54,6.250874e-27,20.228615,True,potassium ion transmembrane transport,biological_process
GO:0098660,53,366,9.469229e-27,6.327245,True,inorganic ion transmembrane transport,biological_process
GO:0015079,23,44,1.790297e-26,22.839945,True,potassium ion transmembrane transporter activity,molecular_function
GO:0006813,25,56,1.982589e-26,19.506164,True,potassium ion transport,biological_process


In [154]:
merged_go_counts_with_descriptions.sort_values('p_val').to_csv('go_enrichment_results.csv')